In [1]:
import MySQLdb as mdb
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import timezone

In [2]:
# Create database connection
db_host = 'localhost'
db_user = 'crypto_user'
db_pass = 'Blue0811'
db_name = 'cryptocurrencies_master'
db_connection = mdb.connect(host=db_host, user=db_user,
                            passwd=db_pass, db=db_name)

In [3]:
# Extract cryptocurrency trading pairs from price_1min table
with db_connection.cursor() as cur:
    cur.execute('SELECT DISTINCT id FROM price_1min;')
    crypto_pairs = list(cur.fetchall())
print('Crypto-Pairs from 1-minute Data Table:')
for pair in crypto_pairs:
    print(pair[0])

Crypto-Pairs from 1-minute Data Table:
BCH-USD
BTC-USD
ETH-USD
LTC-USD


In [4]:
# Define function to transform unix timestamps to trading day indicies
def stamp_to_trading_day(timestamp):
    date = datetime.fromtimestamp(timestamp).astimezone(timezone('UTC'))
    minute = date.hour*60 + date.minute
    return (date.month, date.day, minute)

In [7]:
# Obtain volume data for each currency fom database
unix_start = 1577836800
unix_end = 1585008000
crypto_data = {}

for pair in crypto_pairs:
    pair = pair[0]
    with db_connection.cursor() as cur:
        execution_str = 'SELECT price_date, volume \nFROM price_1min \nWHERE id = "{}" AND '.format(pair)
        execution_str += '(price_date >= {} AND price_date < {})\n'.format(unix_start, unix_end)
        execution_str += 'ORDER BY price_date;'
        print('query:\n{}\n'.format(execution_str))
        cur.execute(execution_str)
        data = list(cur.fetchall())
    crypto_data[pair] = pd.DataFrame(data, columns=('price_date', 'volume'))
    crypto_data[pair]['price_date'] = crypto_data[pair]['price_date'].apply(stamp_to_trading_day)
    crypto_data[pair] = crypto_data[pair].set_index('price_date')

query:
SELECT price_date, volume 
FROM price_1min 
WHERE id = "BCH-USD" AND (price_date >= 1577836800 AND price_date < 1585008000)
ORDER BY price_date;

query:
SELECT price_date, volume 
FROM price_1min 
WHERE id = "BTC-USD" AND (price_date >= 1577836800 AND price_date < 1585008000)
ORDER BY price_date;

query:
SELECT price_date, volume 
FROM price_1min 
WHERE id = "ETH-USD" AND (price_date >= 1577836800 AND price_date < 1585008000)
ORDER BY price_date;

query:
SELECT price_date, volume 
FROM price_1min 
WHERE id = "LTC-USD" AND (price_date >= 1577836800 AND price_date < 1585008000)
ORDER BY price_date;



In [8]:
# Define the trading day indicies
trading_days = []
for m, n in ((1, 31), (2, 29), (3, 23)):
    trading_days += [(m, i) for i in range(1, n + 1)]
    
volume_cryptos = pd.DataFrame(columns=trading_days, index=range(0, 1440))

In [9]:
# Define function to compute the average volume for each trading day and minute 
def compute_vol_avg(trading_stamp, crypto_data):
    total_vol = 0
    for pair in crypto_data.keys():
        if crypto_data[pair].index.contains(trading_stamp):
            total_vol += crypto_data[pair]['volume'][trading_stamp]
        else:
            total_vol += 0
    return total_vol / len(crypto_data.keys())

In [10]:
# Compute volume data for the volume_cryptos table
for day in volume_cryptos.columns:
    for minute in volume_cryptos.index:
        trading_stamp = day + (minute,)
        volume_cryptos[day][minute] = compute_vol_avg(trading_stamp, crypto_data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  """


In [12]:
print('Data Shape: {}'.format(volume_cryptos.shape))
volume_cryptos.head()

Data Shape: (1440, 83)


,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)","(1, 6)","(1, 7)","(1, 8)","(1, 9)","(1, 10)",...,"(3, 14)","(3, 15)","(3, 16)","(3, 17)","(3, 18)","(3, 19)","(3, 20)","(3, 21)","(3, 22)","(3, 23)"
0,15.4148,5.74462,9.84048,133.17,231.889,14.1383,54.1864,294.255,30.4932,3.63267,...,152.932,94.3192,75.45,1447.1,1165.97,300.085,700.473,51.8646,147.664,297.986
1,39.561,13.6749,6.09118,29.7348,367.953,1.36042,8.72272,60.3593,26.2082,2.83987,...,0,53.7927,302.534,597.365,227.328,2080.33,200.396,108.703,197.378,0
2,34.7409,6.07106,7.77497,158.906,164.252,2.72117,32.6313,49.8616,25.4066,20.7674,...,234.486,203.519,1660.25,157.184,250.954,276.849,146.67,332.711,100.294,0
3,77.4941,166.328,92.7528,23.981,8.8465,11.1614,112.987,111.448,16.8685,9.26806,...,142.891,647.077,553.666,188.07,124.122,368.93,279.159,778.891,17.6822,0
4,31.8432,1.64783,149.396,221.861,17.4676,53.9823,47.3113,211.829,9.91072,5.41136,...,375.477,118.862,182.785,311.361,144.336,172.522,155.211,192.128,51.8341,0


In [14]:
# Compute volume data for the volume_cryptos table
volume_days = pd.DataFrame(columns=crypto_data.keys(), index=range(1440))
minutes = range(24*60)
trading_minutes = {minute: [day + (minute,) for day in trading_days] for minute in minutes}
for pair in crypto_data.keys():
    for minute in minutes:
        volume_days[pair][minute] = crypto_data[pair]['volume'][trading_minutes[minute]].mean()

/home/ryan/.local/lib/python3.6/site-packages/pandas/core/series.py:1152: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [15]:
print('Data Shape: {}'.format(volume_days.shape))
volume_days.head()

Data Shape: (1440, 4)


,BCH-USD,BTC-USD,ETH-USD,LTC-USD
0,93.0835,34.1376,340.592,505.784
1,80.4298,34.4632,274.985,400.732
2,69.4601,27.0749,302.014,342.103
3,52.5369,26.3054,259.084,234.649
4,52.0044,18.6781,198.096,211.271


In [16]:
# Write volume data tables to pickle files
volume_cryptos.to_pickle('./volume_cryptos.pkl')
volume_days.to_pickle('./volume_days.pkl')